In [2]:
import numpy as np
import os
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns

# sns.set(1.3)

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.regularizers import l2


# VGG16 모델 로드 (include_top=False로 특징 추출용으로 사용)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# 모델 구성
model = Sequential()
model.add(base_model)

# 3차원 출력 -> 1차원 벡터로 변환 (Flatten 또는 GlobalAveragePooling2D 중 하나 사용)
model.add(GlobalAveragePooling2D())  # Flatten 대신 GlobalAveragePooling2D 사용 가능
# model.add(Flatten())  # 또는 Flatten 사용 가능

model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))  # L2 정규화 추가, Dense 레이어 크기 조정
model.add(BatchNormalization())  # Batch Normalization 추가
model.add(Dropout(0.5))  # Dropout 비율을 0.5로 유지
model.add(Dense(1, activation='sigmoid'))  # 이진 분류를 위한 출력층

# 기존 VGG16의 가중치를 고정 (마지막 6개 레이어만 학습 가능)
base_model.trainable = True
for layer in base_model.layers[:-6]:
    layer.trainable = False

# 모델 컴파일 (학습률 낮춤)
model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=1e-5),  # 학습률 1e-5로 조정
              metrics=['accuracy'])

# 이미지 데이터를 0~1 사이 값으로 스케일링
train_datagen = ImageDataGenerator(rescale=1/255)

# train set
train_generator = train_datagen.flow_from_directory(
    './data/train_added/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

# valid set
valid_generator = train_datagen.flow_from_directory(
    './data/validation-horse-or-human/',
    target_size=(224, 224),
    batch_size=64,
    class_mode='binary'
)

# 콜백 설정
es = EarlyStopping(
    monitor='val_loss',
    patience=10,  # patience 값을 10으로 증가
    verbose=1
)

checkpoint = ModelCheckpoint(
    filepath='./best_model.keras',
    monitor='val_loss',
    verbose=2,
    save_best_only=True,
    mode='auto'
)

# 학습률 조정 콜백 (ReduceLROnPlateau)
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

# steps_per_epoch 설정
steps_per_epoch = train_generator.samples // train_generator.batch_size
validation_steps = valid_generator.samples // valid_generator.batch_size

# 모델 학습
history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=validation_steps,
    epochs=20,  # 더 많은 에포크
    verbose=1,
    callbacks=[checkpoint, es, reduce_lr]
)

Found 2890 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
Epoch 1/20


c:\Users\xorud\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.7024 - loss: 0.8000
Epoch 1: val_loss improved from inf to 0.75796, saving model to ./best_model.keras
45/45 ━━━━━━━━━━━━━━━━━━━━ 232s 5s/step - accuracy: 0.7048 - loss: 0.7957 - val_accuracy: 0.8438 - val_loss: 0.7580 - learning_rate: 1.0000e-05
Epoch 2/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9062 - loss: 0.4200 - learning_rate: 1.0000e-05
Epoch 3/20


c:\Users\xorud\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)
c:\Users\xorud\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\callbacks\model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
c:\Users\xorud\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\callbacks\early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
c:\Users\xorud\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\callbacks\callback_list.py:96: UserWarning: Learning r

45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9502 - loss: 0.3624
Epoch 3: val_loss improved from 0.75796 to 0.67275, saving model to ./best_model.keras
45/45 ━━━━━━━━━━━━━━━━━━━━ 233s 5s/step - accuracy: 0.9505 - loss: 0.3618 - val_accuracy: 0.9727 - val_loss: 0.6728 - learning_rate: 1.0000e-05
Epoch 4/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 341us/step - accuracy: 1.0000 - loss: 0.2803 - learning_rate: 1.0000e-05
Epoch 5/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9937 - loss: 0.2660
Epoch 5: val_loss improved from 0.67275 to 0.59540, saving model to ./best_model.keras
45/45 ━━━━━━━━━━━━━━━━━━━━ 221s 5s/step - accuracy: 0.9936 - loss: 0.2659 - val_accuracy: 1.0000 - val_loss: 0.5954 - learning_rate: 1.0000e-05
Epoch 6/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 5s 341us/step - accuracy: 1.0000 - loss: 0.2363 - learning_rate: 1.0000e-05
Epoch 7/20
45/45 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.9917 - loss: 0.2474
Epoch 7: val_loss improved from 0.59540 to 0.50934, saving model to ./be

In [3]:
# 모델 검증
results = model.evaluate(valid_generator)
print("test loss, test acc:", results)

4/4 ━━━━━━━━━━━━━━━━━━━━ 14s 3s/step - accuracy: 0.9885 - loss: 0.2543
test loss, test acc: [0.2562658488750458, 0.9921875]


In [4]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy & loss')
plt.xlabel('Epoch')
plt.ylabel('accuracy & loss')
plt.legend(['accuracy', 'loss', 'val_loss', 'val_accuracy'], loc='center right')
plt.show()

NameError: name 'history' is not defined

In [6]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 33,791,557 (128.90 MB)

 Trainable params: 9,505,281 (36.26 MB)

 Non-trainable params: 5,275,712 (20.13 MB)

 Optimizer params: 19,010,564 (72.52 MB)